__title__      = ML in Action Chapter9 Code（regTrees）
__author__     = wgj
__createDate__ = 2018-11-05 20:30:28

In [17]:
from numpy import *

def loadDataSet(fileName): 
    dataMat = []                           #  最后1列为目标值
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')         # 移除每行首尾空格并用tab分割数据
        fltLine = list(map(float,curLine))     # 将每行映射成浮点数(Python3 需在map前加list)
        dataMat.append(fltLine)
    return dataMat

""" 对数据集进行2向切分 """
def binSplitDataSet(dataSet, feature, value):
    """ 
    输入：1、dataSet——数据集（m行xn列）
                2、feature——选择用来切分的特征（列索引）
                3、value——特征值
    输出：1、mat0——数据集在特征feature下 > value的分支
                2、mat1——数据集在特征feature下<= value的分支
    """
    mat0 = dataSet[nonzero(dataSet[:,feature] > value)[0],:]
    mat1 = dataSet[nonzero(dataSet[:,feature] <= value)[0],:]
#     mat0 = dataSet[dataSet[:,feature] > value,:]        # 仅能用于array
#     mat1 = dataSet[dataSet[:,feature] <= value,:]
    return mat0,mat1


In [15]:
""" 计算目标变量均值 """
def regLeaf(dataSet):#returns the value used for each leaf
    return mean(dataSet[:,-1])

""" 计算目标变量总方差 """
def regErr(dataSet):
    return var(dataSet[:,-1]) * shape(dataSet)[0]

""" 选择最佳切分特征 """
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    """ 
    输入：1、dataSet——数据集（m行xn列）
                2、leafType——建立叶节点函数的引用
                3、errType——误差计算函数的引用
                4、ops——其他参数（元组）
    输出：1、bestIndex——最佳用来切分的特征（列索引）
                2、bestValue——特征值
    """
    tolS = ops[0];    # 容许的误差下降值
    tolN = ops[1]    # 切分出的最小样本数
    
    # 如果剩余1个特征则退出（1）
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1: 
        return None, leafType(dataSet)
    m,n = shape(dataSet)
    #the choice of the best feature is driven by Reduction in RSS error from mean
    S = errType(dataSet)                                            # 当前数据集的误差
    bestS = inf; bestIndex = 0; bestValue = 0         # 初始化最小误差、最佳特征、最佳特征值
    for featIndex in range(n-1):                               # 遍历所有特征
        for splitVal in set(dataSet[:,featIndex].T.A.tolist()[0]):        # 遍历该特征的所有值
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)                 # 进行切分
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN): continue      # 跳过样本数<阈值的切分
            newS = errType(mat0) + errType(mat1)                                                 # 计算当前切分下的两组样本集的误差和
            if newS < bestS:                                            # 保留误差较小的分割结果
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
                # 最佳切分即使得切分后能达到最低误差的切分

    # 误差减小量小于阈值则退出（2）
    if (S - bestS) < tolS: 
        return None, leafType(dataSet) 
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    
    # 如果切分出的数据集小于阈值则退出（3）
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN): 
        return None, leafType(dataSet)
    return bestIndex,bestValue

""" 构建树 """
def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):#数据集为NumPy Mat可进行索引
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)  # 选择最佳切分
    if feat == None: return val #if the splitting hit a stop condition return val
    retTree = {}                            # 初始化树为空字典
    retTree['spInd'] = feat          # 树节点所对应的特征
    retTree['spVal'] = val             #  树节点所对应的特征值
    lSet, rSet = binSplitDataSet(dataSet, feat, val)                        # 按照选择的特征进行数据集切分
    retTree['left'] = createTree(lSet, leafType, errType, ops)        # 构建左子树
    retTree['right'] = createTree(rSet, leafType, errType, ops)     # 构建右子树
    return retTree

测试回归树构建函数

In [18]:
dat1 = loadDataSet('ex0.txt')
dat1 = mat(dat1)
createTree(dat1)

{'spInd': 1,
 'spVal': 0.39435,
 'left': {'spInd': 1,
  'spVal': 0.582002,
  'left': {'spInd': 1,
   'spVal': 0.797583,
   'left': 3.9871632,
   'right': 2.9836209534883724},
  'right': 1.980035071428571},
 'right': {'spInd': 1,
  'spVal': 0.197834,
  'left': 1.0289583666666666,
  'right': -0.023838155555555553}}

模型树相关函数

In [24]:
def linearSolve(dataSet):   #helper function used in two places
    m,n = shape(dataSet)
    X = mat(ones((m,n))); Y = mat(ones((m,1)))#create a copy of data with 1 in 0th postion
    X[:,1:n] = dataSet[:,0:n-1]; Y = dataSet[:,-1]#and strip out Y
    xTx = X.T*X
    if linalg.det(xTx) == 0.0:
        raise NameError('This matrix is singular, cannot do inverse,\n\
        try increasing the second value of ops')
    ws = xTx.I * (X.T * Y)
    return ws,X,Y

def modelLeaf(dataSet):#create linear model and return coeficients
    ws,X,Y = linearSolve(dataSet)
    return ws

def modelErr(dataSet):
    ws,X,Y = linearSolve(dataSet)
    yHat = X * ws
    return sum(power(Y - yHat,2))

树回归与标准回归的比较

In [22]:
def isTree(obj):
    return (type(obj).__name__=='dict')

""" 对回归树叶节点进行预测 """
def regTreeEval(model, inDat):
    return float(model)                # 回归树的叶节点即为均值

""" 对模型树叶节点进行预测 """
def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1,n+1)))
    X[:,1:n+1]=inDat              # 与上1行的共同作用是对原始数据集前增加1个1
    return float(X*model)   # X与系数W的乘积即为预测值

""" 自顶向下遍历树，返回叶节点预测值 """
def treeForeCast(tree, inData, modelEval=regTreeEval):
    """ 
    输入：1、tree——树结构
                2、inData——输入数据（单点或行向量）
                3、modelEval——对叶节点数据进行预测函数的引用
    输出：叶节点预测值
    """
    if not isTree(tree): return modelEval(tree, inData)    # 叶子节点直接返回预测值
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)

""" 对一组测试数据集计算预测值 """
def createForeCast(tree, testData, modelEval=regTreeEval):
    """ 
    输入：1、tree——树结构
                2、testData——测试数据集
                3、modelEval——对叶节点数据进行预测函数的引用
    输出：yHat——一组预测值（行向量）
    """
    m = len(testData)
    yHat = mat(zeros((m,1)))
    for i in range(m):
        yHat[i,0] = treeForeCast(tree, mat(testData[i]), modelEval)
    return yHat

进行比较

In [23]:
""" 创建1颗回归树 """
trainMat = mat(loadDataSet('bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('bikeSpeedVsIq_test.txt'))
myTree = createTree(trainMat, ops=(1,20))
yHat = createForeCast(myTree, testMat[:,0])
corrcoef(yHat, testMat[:,1], rowvar=0)[0,1]


0.9640852318222145

In [25]:
""" 创建1颗模型树 """
myTree = createTree(trainMat, modelLeaf, modelErr,ops=(1,20))
yHat = createForeCast(myTree, testMat[:,0], modelTreeEval)
corrcoef(yHat, testMat[:,1], rowvar=0)[0,1]

0.9760412191380623

In [27]:
""" 标准线性回归 """
ws, X, Y = linearSolve(trainMat)
for i in range(shape(testMat)[0]):
    yHat[i] = testMat[i,0]*ws[1,0]+ws[0,0]
corrcoef(yHat, testMat[:,1], rowvar=0)[0,1]

0.9434684235674762

由上述结果可知，在相关系数度量下，三种方法优略如下：
标准线性回归<回归树<模型树